In [4]:
import pandas as pd
dat = pd.read_csv('abl.csv')
dat = dat.iloc[2000:3000,:]

In [5]:
# retrieving location: longitude and latitude based on city and state
from geopy import Nominatim
import numpy as np
geolocator = Nominatim(user_agent="app")
dat["FULL_LOC"] = dat["CITY"] + " " + dat["STATE OR COUNTRY NAME"]

loc = list(dat["FULL_LOC"].unique())
len(loc)

65

In [7]:
geo = [geolocator.geocode(i) for i in loc] 

In [30]:
loc[geo.index(None)]

'WASHINGTON DIST OF COL'

In [27]:
geolocator.geocode("WASHINGTON DISTRICT OF COLUMBIA")

Location(Washington, District of Columbia, United States, (38.8950368, -77.0365427, 0.0))

In [11]:
import numpy as np
lat = [i.latitude if i is not None else np.nan for i in geo ]
lon = [i.longitude if i is not None else np.nan for i in geo ]

In [31]:
lat_dict = dict(zip(loc,lat))
lon_dict = dict(zip(loc,lon))

lat_dict.update({'WASHINGTON DIST OF COL':38.8950368})
lon_dict.update({'WASHINGTON DIST OF COL': -77.0365427})

In [93]:
dat["LON"] = dat["FULL_LOC"].map(lon_dict).astype(int)
dat["LAT"] = dat["FULL_LOC"].map(lat_dict).astype(int)

In [94]:
dat[dat["LON"].isna()]

,ORGANIZATION NAME,ORGANIZATION ID (IPF),PROJECT NUMBER,FUNDING MECHANISM,NIH REFERENCE,PI NAME,PI PERSON ID,PROJECT TITLE,DEPT NAME,NIH DEPT COMBINING NAME,...,ATTRIBUTED TO MEDICAL SCHOOL,MEDICAL SCHOOL LOCATION,INSTITUTION TYPE,AWARD NOTICE DATE,FOA,FULL_LOC,LON,LAT,MONTH,YEAR


In [38]:
dat["MONTH"] = pd.to_datetime(dat["AWARD NOTICE DATE"]).dt.month
dat["YEAR"] = pd.to_datetime(dat["AWARD NOTICE DATE"]).dt.year

In [48]:
dat["FUNDING MECHANISM"].unique() 

['RPGs - Non SBIR/STTR',
 'Other Research-Related',
 'Training - Individual',
 'RPGs - SBIR/STTR',
 'Training - Institutional',
 'Research Centers',
 'Other',
 'all']

In [41]:
dat["INSTITUTION TYPE"].unique()

array(['Domestic Higher Education', nan, 'Research Institutes',
       'Independent Hospitals', 'None'], dtype=object)

In [44]:
dat["FUNDING"].head()

2000    $249,901 
2001    $298,587 
2002     $90,914 
2003     $77,070 
2004    $422,500 
Name: FUNDING, dtype: object

In [97]:
dat.to_csv("abl-short.csv")

In [98]:

# dash
import dash
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

# plotting and df
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import pandas as pd 


# load data
# for now we just read pre-saved file
proj = pd.read_csv("abl-short.csv").dropna()
years = list(proj["YEAR"].unique()) 
minyr = min(years)
maxyr = max(years)
years = years + ["all"]
months = list(proj["MONTH"].unique()) + ["all"]
funds = list(proj["FUNDING MECHANISM"].unique()) + ["all"]
insts = list(proj["INSTITUTION TYPE"].unique()) + ["all"]



In [99]:
access = "pk.eyJ1IjoibGlubmlsIiwiYSI6ImNsZW01bW9vaTBhMjIzdm1sb2V1b25qemcifQ.W5w-j8UQOorI_ZbO2ibrzA"
style = "mapbox://styles/linnil/clem66ou8000w01qus6lbbdm9"


In [102]:
#https://plotly.com/python-api-reference/generated/plotly.express.scatter_mapbox.html
fig = px.scatter_mapbox(
            proj,
            lat="LAT",
            lon="LON",
            hover_name="FULL_LOC",
            hover_data=["INSTITUTION TYPE","FUNDING MECHANISM","FUNDING"],
            #marker=dict(size=5, color="white", opacity=0),
            zoom=3
        # color = new['count'], color by fund amount
)
fig.update_mapboxes(
    accesstoken = "pk.eyJ1IjoibGlubmlsIiwiYSI6ImNsZW01bW9vaTBhMjIzdm1sb2V1b25qemcifQ.W5w-j8UQOorI_ZbO2ibrzA",
    style="mapbox://styles/linnil/clem66ou8000w01qus6lbbdm9"
)
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()